# Cодержание  
 - 1. Загрузка и изучение данных  
 - 2. Обучение и проверка модели
 - 3. Подготовка к расчёту прибыли
 - 4. Расчёт прибыли и рисков
 - 5. Общий вывод

# Выбор локации для скважины
Нужно решить, где бурить новую скважину.  
Шаги для выбора локации обычно такие:  
В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;  
Строят модель для предсказания объёма запасов в новых скважинах;  
Выбирают скважины с самыми высокими оценками значений;  
Определяют регион с максимальной суммарной прибылью отобранных скважин.  
Нам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Необходимо построить модель для определения региона, где добыча принесёт наибольшую прибыль.   
Возможную прибыль и риски нужно проанализировать техникой Bootstrap.

**Описание данных:**  
id — уникальный идентификатор скважины;  
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);  
product — объём запасов в скважине (тыс. баррелей).  
**Условия задачи:**  
Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).  
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.  
Бюджет на разработку скважин в регионе — 10 млрд рублей.  
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.  
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.  

## Загрузка и изучение данных

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

Поскольку датасеты уже приведены к общему виду, на данном этапе будем рассматривать их вместе, для экономии времени.

In [2]:
try:
    df1 = pd.read_csv('geo_data_0.csv')
    df2 = pd.read_csv('geo_data_1.csv')
    df3 = pd.read_csv('geo_data_2.csv')
except:
    df1 = pd.read_csv('/datasets/geo_data_0.csv')
    df2 = pd.read_csv('/datasets/geo_data_1.csv')
    df3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Пропусков нет, названия столбцов и числовые форматы признаков нас устраивают.

In [6]:
df1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [7]:
df2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [8]:
df3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Можно отметить, что все признаки являются количественными.

In [9]:
df1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [10]:
df2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [11]:
df3.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Минусовых значений запаса продукта не обнаружено, других явных аномалий также не обнаружено.

In [12]:
display(df1.duplicated().sum())
display(df2.duplicated().sum())
display(df3.duplicated().sum())

0

0

0

Полных дубликатов тоже нет.  
Предобработка данных не требуется, можно переходить к обучению модели.

## Обучение и проверка модели

В условиях задачи заложено использование линейной регрессии, поэтому другие модели трогать не будем.

In [13]:
model = LinearRegression()
# скейлер для масштаюирования признаков, лишним не будет
scaler = StandardScaler()

Столбец 'id' в расчетах нам не понадобится, его просто отбрасываем.  
Целевым признаком является 'product'.

Разбиваем данные на обучающую и валидационную выборки в соотношении 3 к 1.  
Масштабируем признаки при помощи StandartScaler.  
Выводим среднее значение предсказания модели и RMSE.  
Выполним эти действия для трех датасетов и сравним результаты.

Я не стал создавать отдельные переменные для признаков по каждому из датасетов, поскольку мы работаем с одной и той же моделью и удобнее просто перезаписать их, не меняя код основной части обучения и проверки.

In [14]:
features = df1.drop(columns={'id', 'product'})
target = df1['product']

In [15]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions, squared=False)
display(predictions.mean())
display('RMSE:', rmse)

92.59256778438035

'RMSE:'

37.5794217150813

In [16]:
features = df2.drop(columns={'id', 'product'})
target = df2['product']

In [17]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions, squared=False)
display(predictions.mean())
display('RMSE:', rmse)

68.728546895446

'RMSE:'

0.8930992867756167

In [18]:
features = df3.drop(columns={'id', 'product'})
target = df3['product']

In [19]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions, squared=False)
display(predictions.mean())
display('RMSE:', rmse)

94.96504596800489

'RMSE:'

40.02970873393434

Результаты по первому и третьему региону в целом похожи.  
По второму региону модель работает значительно точнее, однако средний уровень предсказанных запасов также сильно ниже.  
Дальнейшие выводы по качеству модели для каждого региона можно будет сделать только после подсчета прибыли и рисков.

## Подготовка к расчёту прибыли

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.  
На разработку 200 месторождений в регионе выделяется 10 миллиардов рублей.  
Обладая этими данными, вычислим минимальный запас сырья для безубыточной разработки месторождения и затем сравним его с средними запасами в каждом регионе.

In [20]:
UNIT_INCOME = 450000
REGION_BUDGET = 10000000000
SINGLE_BUDGET = REGION_BUDGET / 200
MINIMAL_PRODUCT = SINGLE_BUDGET / UNIT_INCOME
display('минимальный объем сырья необходимый для безубыточности:', MINIMAL_PRODUCT)
display('средний запас сырья в первом регионе:', df1['product'].mean())
display('средний запас сырья во втором регионе:', df2['product'].mean())
display('средний запас сырья в третьем регионе:', df3['product'].mean())

'минимальный объем сырья необходимый для безубыточности:'

111.11111111111111

'средний запас сырья в первом регионе:'

92.49999999999974

'средний запас сырья во втором регионе:'

68.82500000002561

'средний запас сырья в третьем регионе:'

95.00000000000041

Ни в одном регионе, средний запас сырья не превышает минимальный необходимый для безубыточности месторождения.  
Это говорит нам о важности дополнительных средств оценки потенциала, в том числе машинного обучения.  
Подготовим функцию для расчета прибыли:

Поскольку нам известно что при разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки, функция будет отбирать 200 самых богатых месторождений по предсказанию модели, затем по их идексам берутся настоящие данные из столбца 'product', которые находятся в переменной с целевым признаком, эти данные суммируются и из них вычитаем 10 млрд. бюджета.

In [21]:
def profit_check(target, predictions):
    pred_sorted = predictions.nlargest(n=200)
    selected = target[pred_sorted.index]
    profit = selected.sum() * UNIT_INCOME - 10000000000
    return profit

## Расчёт прибыли и рисков

Для того, чтобы посмотреть распределение прибыли, также оценить вероятность убытков, применим технику Boostrap.

In [22]:
# Случайное значение для Boostrap
state = np.random.RandomState(12345)
sample_size = 500

In [23]:
features = df1.drop(columns={'id', 'product'})
target = df1['product']
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
target_valid = pd.Series(target_valid).reset_index(drop=True)
predictions = pd.Series(predictions).reset_index(drop=True)
profits = []
for i in range(1000):
    target_subsample = target_valid.sample(n=500, random_state=state)
    predictions_subsample = predictions[target_subsample.index]
    profits.append(profit_check(target_subsample, predictions_subsample))
profits = pd.Series(profits)
display('средняя прибыль:', profits.mean())
display('вероятность убытка:', (profits < 0).mean())
display('95% доверительный интервал:' , profits.quantile(0.025), profits.quantile(0.975))

'средняя прибыль:'

380710890.70907456

'вероятность убытка:'

0.072

'95% доверительный интервал:'

-126947638.03180212

879613967.8477957

In [24]:
features = df2.drop(columns={'id', 'product'})
target = df2['product']
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
target_valid = pd.Series(target_valid).reset_index(drop=True)
predictions = pd.Series(predictions).reset_index(drop=True)
profits = []
for i in range(1000):
    target_subsample = target_valid.sample(n=500, random_state=state)
    predictions_subsample = predictions[target_subsample.index]
    profits.append(profit_check(target_subsample, predictions_subsample))
profits = pd.Series(profits)
display('средняя прибыль:', profits.mean())
display('вероятность убытка:', (profits < 0).mean())
display('95% доверительный интервал:' , profits.quantile(0.025), profits.quantile(0.975))

'средняя прибыль:'

454785434.76571316

'вероятность убытка:'

0.013

'95% доверительный интервал:'

46730084.769206285

840213356.26002

In [25]:
features = df3.drop(columns={'id', 'product'})
target = df3['product']
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
target_valid = pd.Series(target_valid).reset_index(drop=True)
predictions = pd.Series(predictions).reset_index(drop=True)
profits = []
for i in range(1000):
    target_subsample = target_valid.sample(n=500, random_state=state)
    predictions_subsample = predictions[target_subsample.index]
    profits.append(profit_check(target_subsample, predictions_subsample))
profits = pd.Series(profits)
display('средняя прибыль:', profits.mean())
display('вероятность убытка:', (profits < 0).mean())
display('95% доверительный интервал:' , profits.quantile(0.025), profits.quantile(0.975))

'средняя прибыль:'

389217073.6855697

'вероятность убытка:'

0.073

'95% доверительный интервал:'

-115609565.77767342

906512590.1730093

## Общий вывод
Предоставленные данные не требовали предобработки.  
Данные из столбца 'id' были отброшены за ненадобностью для выполнения задачи.  
Заявленная по условию задачи модель(линейная регрессия) дала наиболее точные прогнозы по второму региону.  
Была написана формула для для расчёта прибыли по выбранным скважинам и предсказаниям модели.  
С применением этой формулы и метода Boostrap, были посчитаны прибыль и риски для каждого региона.  
Во втором регионе, несмотря на самый низкий средний уровень запасов сырья, прогнозируется наибольшая прибыль, а также наименьший шанс убытка(ниже требуемых 2.5%).  
К разработке рекоммендуется второй регион, так как, это регион, на данных которого, модель показала наилучший RMSE, и вполне вероятно, что высокая прогнозируемая прибыль и низкая вероятность убытков обусловлена именно точностью предсказаний.  